<a href="https://colab.research.google.com/github/zherenz/PyTorch-Basics-22/blob/main/torch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset & DataLoader<br>
torch.utils.data.DataLoader<br>
torchvision.datasets<br>
torchvision.datasets.ImageFolder<br>

## torchvision.datasets

In [ ]:
from torchvision import datasets

# torchvision.datasets.FashionMNIST
train_data = datasets.FashionMNIST(root="data", train=True, download=False, transform=ttf.ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=False, transform=ttf.ToTensor())

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# torchvision.datasets.CIFAR10
train_set = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/deep_learning/data', train=True, download=False, transform=ttf.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)


# torchvision.datasets.ImageNet(root: str, split: str = 'train', **kwargs: Any)
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)

# We have loaded that dataset into the DataLoader and can iterate through the dataset as needed. 
# Each iteration below returns a batch of train_features and train_labels (containing batch_size=64 features and labels respectively).

## torchvision.datasets.ImageFolder

In [ ]:
import torch
import torchvision
import torchvision.transforms as ttf

train_data = torchvision.datasets.ImageFolder(train_path, transform=ttf.ToTensor())
val_data = torchvision.datasets.ImageFolder(val_path, transform=ttf.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_worker=4, shuffle=False)

In [ ]:
train_transforms = [ttf.ToTensor(), ttf.RandomHorizontalFlip(), ttf.RandomAffine(degrees=(-15, 15), scale=(0.98, 1.03)), ttf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
val_transforms = [ttf.ToTensor(), ttf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]

# root/dog/xxx.png
# root/dog/xxy.png
# root/dog/[...]/xxz.png

# root/cat/123.png
# root/cat/nsdf3.png
# root/cat/[...]/asd932_.png

# Model<br>
torch.nn.Module<br>
torch.nn.Sequential<br>

## torch.nn.Module (LeNet) <br>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = torch.flatten(self.num_flat_features(x), start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


model = Model()
print(model)

## torch.nn.Sequential

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(3, 32, 3),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    
    nn.Conv2d(32, 64, 3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    # print the size of the output --> output.size() 
    # to find out the input size of fc 
    
    # flatten from 2nd dimesion (batch)
    nn.Flatten(),
    nn.Linear(64 * 6 * 6, 4096),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(4096, 10)
)

print(model)

# Loss Function

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

criterion = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
criterion = torch.nn.L1Loss(size_average=None, reduce=None, reduction='mean')
criterion = torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
criterion = torch.nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean', beta=1.0)

## BCE-Dice Loss

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = F.sigmoid(inputs)       
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()                     
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

# Train & Eval